In [2]:
import pandas as pd
import yfinance as yf
from pathlib import Path

In [6]:
universe_path = Path("./Universe/Universe.csv")

universe = pd.read_csv(universe_path)

# Sanity check
print(universe.head())
print(universe.columns)

  Ticker                                          Name         Category  \
0    SPY                        SPDR S&P 500 ETF Trust   US Large Blend   
1    IVV                      iShares Core S&P 500 ETF   US Large Blend   
2    VOO                          Vanguard S&P 500 ETF   US Large Blend   
3    VTI               Vanguard Total Stock Market ETF  US Total Market   
4   ITOT  iShares Core S&P Total U.S. Stock Market ETF  US Total Market   

  Factor_bucket                                              Notes  \
0          Core  Flagship S&P 500 tracker widely used as refere...   
1          Core                 S&P 500 low-cost, core large caps.   
2          Core                 Alternative Vanguard S&P 500 core.   
3          Core  Broad US equity market including small and mid...   
4          Core     Total market US, univers large pour beta core.   

  Inception_date  
0     1993-01-22  
1     2000-05-15  
2     2010-09-07  
3     2000-11-13  
4     2004-01-20  
Index(['Ticker

In [7]:
# Parse inception dates (adapt format if your file uses another one)
universe['Inception_date'] = pd.to_datetime(universe['Inception_date'], errors='coerce')

# Vérifier s'il reste des NaT
print(universe[['Ticker', 'Inception_date']].head(20))

   Ticker Inception_date
0     SPY     1993-01-22
1     IVV     2000-05-15
2     VOO     2010-09-07
3     VTI     2000-11-13
4    ITOT     2004-01-20
5    SCHB     2009-11-03
6     IWB     2000-05-15
7     RSP     2003-04-24
8    USMF     2017-06-29
9    JPUS     2015-09-29
10   OMFL     2017-11-08
11   DGRO     2014-06-10
12   SCHD     2011-10-20
13    VIG     2006-04-21
14    DVY     2003-11-03
15    HDV     2011-03-29
16   NOBL     2013-10-09
17    SDY     2005-11-08
18   RDVY     2014-01-06
19   USMV     2011-10-18


In [8]:
# on commence à la min des inception dates → certains fonds auront des NaN avant leur lancement.
global_start = universe['Inception_date'].min()

global_end = pd.to_datetime("today").normalize()

print("Global start:", global_start.date())
print("Global end  :", global_end.date())

Global start: 1993-01-22
Global end  : 2025-11-16


In [13]:
# Fonction utilitaire pour télécharger les prix mensuels
# On utilise interval='1mo'

def download_monthly_prices(ticker: str, start: pd.Timestamp, end: pd.Timestamp) -> pd.Series:
    """
    Download monthly adjusted close for a single ticker using yfinance.
    Returns a pandas Series indexed by date.
    """
    data = yf.download(
        ticker,
        start=start,
        end=end,
        interval="1mo",
        auto_adjust=True,  # use adjusted prices
        progress=False
    )

    if data.empty:
        return pd.Series(dtype=float)

    # Keep only adjusted close
    prices = data['Close'].copy()
    prices.name = ticker
    return prices

In [12]:
data = yf.download(
        "SPY",
        start=global_start,
        end=global_end,
        interval="1mo",
        auto_adjust=True,  # use adjusted prices
        progress=False
    )

print(data.head())

Price           Close       High        Low       Open   Volume
Ticker            SPY        SPY        SPY        SPY      SPY
Date                                                           
1993-01-01  24.313032  24.330324  24.209278  24.330324  1003200
1993-02-01  24.572416  24.970140  23.690507  24.330324  5417600
1993-03-01  25.004721  25.367860  24.468658  24.658874  3019200
1993-04-01  24.479996  25.157583  24.063019  25.157583  2697200
1993-05-01  25.140205  25.383442  24.375749  24.514741  1808000


In [14]:
# Télécharger les prix mensuels pour tout l’univers
tickers = universe['Ticker'].unique().tolist()
print("Number of tickers:", len(tickers))
print(tickers)

price_series_list = []

for t in tickers:
    print(f"Downloading {t}...")
    s = download_monthly_prices(t, start=global_start, end=global_end)
    if s.empty:
        print(f"⚠ No data returned for {t}")
    price_series_list.append(s)

# Concaténer toutes les séries en un DataFrame
prices_monthly = pd.concat(price_series_list, axis=1)

# Ordonner chronologiquement et vérifier
prices_monthly = prices_monthly.sort_index()
prices_monthly.tail()

Number of tickers: 53
['SPY', 'IVV', 'VOO', 'VTI', 'ITOT', 'SCHB', 'IWB', 'RSP', 'USMF', 'JPUS', 'OMFL', 'DGRO', 'SCHD', 'VIG', 'DVY', 'HDV', 'NOBL', 'SDY', 'RDVY', 'USMV', 'SPLV', 'XMLV', 'EFAV', 'LGLV', 'LVHD', 'MTUM', 'QMOM', 'SPMO', 'MOM', 'QUAL', 'SPHQ', 'JQUA', 'SUSA', 'IWM', 'IJR', 'SCHA', 'VO', 'MDY', 'SIZE', 'SMLF', 'VTV', 'IVE', 'IWD', 'VBR', 'IJS', 'VLUE', 'VOOG', 'IWF', 'VUG', 'IWP', 'IWS', 'IUSV', 'IUSG']


Ticker,SPY,IVV,VOO,VTI,ITOT,SCHB,IWB,RSP,USMF,JPUS,...,VBR,IJS,VLUE,VOOG,IWF,VUG,IWP,IWS,IUSV,IUSG
Date,,,,,,,,,,,,,,,,,,,,,
2025-07-01,630.332458,632.992920,579.363098,310.028168,137.724762,24.299858,346.440002,182.834183,50.414825,117.728195,...,197.391296,100.394173,112.337547,410.326630,440.001404,454.695587,141.303528,134.025620,94.945488,155.343704
2025-08-01,643.266602,646.372620,591.388733,317.317902,140.905640,24.868217,353.770538,187.802551,51.673950,121.923042,...,207.874619,109.044037,119.031059,413.602448,445.176605,458.131927,142.792313,138.029343,98.389000,156.711609
2025-09-01,664.338135,667.289551,610.633667,327.260284,145.233215,25.636000,364.512024,188.877869,52.109612,123.229836,...,207.785019,110.079224,124.371948,434.905273,467.975433,479.099579,142.292725,139.104980,99.473801,164.290085
2025-10-01,682.059998,685.229980,627.039978,335.420013,148.960007,26.280001,373.709991,187.929993,50.810001,121.614998,...,206.509995,110.199997,130.410004,450.200012,485.579987,498.850006,142.020004,138.250000,101.029999,169.830002
2025-11-01,671.929993,675.309998,617.830017,329.859985,146.350006,25.820000,367.880005,187.339996,50.799999,122.505997,...,205.460007,110.180000,130.660004,436.529999,470.929993,483.519989,136.839996,137.850006,101.320000,165.000000


In [16]:
# Calculer les rendements mensuels
returns_monthly = prices_monthly.pct_change()

print(returns_monthly.head())

print(returns_monthly.tail())

Ticker           SPY  IVV  VOO  VTI  ITOT  SCHB  IWB  RSP  USMF  JPUS  ...  \
Date                                                                   ...   
1993-01-01       NaN  NaN  NaN  NaN   NaN   NaN  NaN  NaN   NaN   NaN  ...   
1993-02-01  0.010669  NaN  NaN  NaN   NaN   NaN  NaN  NaN   NaN   NaN  ...   
1993-03-01  0.017593  NaN  NaN  NaN   NaN   NaN  NaN  NaN   NaN   NaN  ...   
1993-04-01 -0.020985  NaN  NaN  NaN   NaN   NaN  NaN  NaN   NaN   NaN  ...   
1993-05-01  0.026969  NaN  NaN  NaN   NaN   NaN  NaN  NaN   NaN   NaN  ...   

Ticker      VBR  IJS  VLUE  VOOG  IWF  VUG  IWP  IWS  IUSV  IUSG  
Date                                                              
1993-01-01  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN   NaN   NaN  
1993-02-01  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN   NaN   NaN  
1993-03-01  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN   NaN   NaN  
1993-04-01  NaN  NaN   NaN   NaN  NaN  NaN  NaN  NaN   NaN   NaN  
1993-05-01  NaN  NaN   NaN   NaN  NaN  NaN  NaN  Na

C:\Users\15145\AppData\Local\Temp\ipykernel_24732\3378102776.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_monthly = prices_monthly.pct_change()


In [19]:
# Filtrage et alignement
# On impose un seuil minimal de couverture : au moins 80 % des fonds actifs à cette date.

min_coverage = 0.8
threshold = int(len(tickers) * min_coverage)

mask = returns_monthly.notna().sum(axis=1) >= threshold
returns_filtered = returns_monthly[mask]

print("Original rows:", len(returns_monthly))
print("Filtered rows:", len(returns_filtered))

print(returns_filtered.head())

Original rows: 395
Filtered rows: 148
Ticker           SPY       IVV       VOO       VTI      ITOT      SCHB  \
Date                                                                     
2013-08-01 -0.029992 -0.030374 -0.030817 -0.030313 -0.029942 -0.029283   
2013-09-01  0.026642  0.027372  0.028724  0.033974  0.030198  0.032931   
2013-10-01  0.051406  0.051233  0.049928  0.047708  0.048606  0.047443   
2013-11-01  0.029638  0.029826  0.029960  0.027027  0.030446  0.028005   
2013-12-01  0.020386  0.020279  0.020821  0.021948  0.020260  0.021793   

Ticker           IWB       RSP  USMF  JPUS  ...       VBR       IJS      VLUE  \
Date                                        ...                                 
2013-08-01 -0.023018 -0.029203   NaN   NaN  ... -0.041709 -0.034147 -0.030386   
2013-09-01  0.031774  0.037720   NaN   NaN  ...  0.052975  0.062910  0.022277   
2013-10-01  0.047976  0.046187   NaN   NaN  ...  0.039229  0.037795  0.052541   
2013-11-01  0.027271  0.022209   NaN  

In [20]:
# Sauvegarde des résultats
prices_monthly.to_csv("./output/ETF_monthly_prices.csv", index_label="Date")
returns_monthly.to_csv("./output/ETF_monthly_returns.csv", index_label="Date")
returns_filtered.to_csv("./output/ETF_monthly_returns_filtered.csv", index_label="Date")

print("Files saved:")
print(" - ETF_monthly_prices.csv")
print(" - ETF_monthly_returns.csv")
print(" - ETF_monthly_returns_filtered.csv")

Files saved:
 - ETF_monthly_prices.csv
 - ETF_monthly_returns.csv
 - ETF_monthly_returns_filtered.csv
